# A notebook to copy and clean up PIPS netCDF files for upload to the PERiLS EOL repository

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
import warnings
warnings.simplefilter('ignore')
%matplotlib notebook

In [ ]:
# Read in the original PIPS netcdf files

PIPS_input_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data_for_EOL/'
PIPS_output_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data_for_EOL/'
deployment_name = 'IOP3_040522'
PIPS_input_dir = os.path.join(PIPS_input_base_dir, deployment_name, 'netcdf')
PIPS_output_dir = os.path.join(PIPS_output_base_dir, deployment_name, 'netcdf_temp')
if not os.path.exists(PIPS_output_dir):
    os.makedirs(PIPS_output_dir)

# IOP1
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B']
# IOP2
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
# IOP3
PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
parsivel_interval = 10
parsivel_filenames = ['parsivel_combined_{}_{}_{:d}s.nc'.format(deployment_name, PIPS_name, parsivel_interval)
                      for PIPS_name in PIPS_names]
parsivel_filepaths = [os.path.join(PIPS_input_dir, parsivel_filename) for parsivel_filename in parsivel_filenames]
output_parsivel_filepaths = [os.path.join(PIPS_output_dir, parsivel_filename) 
                             for parsivel_filename in parsivel_filenames]
conv_filenames = ['conventional_raw_{}_{}.nc'.format(deployment_name, PIPS_name) for PIPS_name in PIPS_names]
conv_filepaths = [os.path.join(PIPS_input_dir, conv_filename) for conv_filename in conv_filenames]
output_conv_filepaths = [os.path.join(PIPS_output_dir, conv_filename) for conv_filename in conv_filenames]
parsivel_ds_dict = {}
conv_ds_dict = {}
for PIPS_name, parsivel_filepath, conv_filepath in zip(PIPS_names, parsivel_filepaths, conv_filepaths):
    parsivel_ds_dict[PIPS_name] = xr.load_dataset(parsivel_filepath)
    conv_ds_dict[PIPS_name] = xr.load_dataset(conv_filepath)

In [ ]:
PIPS_to_check = 'PIPS1A'

In [ ]:
parsivel_ds = parsivel_ds_dict[PIPS_to_check]
parsivel_ds

In [ ]:
conv_ds = conv_ds_dict[PIPS_to_check]
conv_ds

In [ ]:
# Make a dictionary of the variable names and their units for those that don't already have them
parsivel_var_units_dict = {
    'windspdavgvec': 'meters per second',
    'winddirunitavgvec': 'degrees',
    'uavg': 'meters per second',
    'vavg': 'meters per second',
    'rho': 'kilograms per cubic meter',
    'fallspeed': 'meters per second',
    'diameter': 'millimeters',
    'min_diameter': 'millimeters',
    'max_diameter': 'millimeters',
    'min_fallspeeds': 'meters per second',
    'max_fallspeeds': 'meters per second',
    'VD_matrix': 'count',
    'ND': 'number per cubic meter per millimeter',
    'VD_matrix_qc': 'count',
    'ND_qc': 'number per cubic meter per millimeter',
    'VD_matrix_roqc': 'count',
    'ND_roqc': 'number per cubic meter per millimeter',
    'pcount_derived': 'count',
    'pcount_derived_qc': 'count',
    'pcount_derived_roqc': 'count'
}

In [ ]:
# Now go through and update the units attribute for the needed variables using this dictionary

for PIPS_name in PIPS_names:
    for var_name in parsivel_ds_dict[PIPS_name].data_vars:
        if var_name in parsivel_var_units_dict:
            unit = parsivel_var_units_dict[var_name]
            parsivel_ds_dict[PIPS_name][var_name].attrs['units'] = unit

In [ ]:
# Now save to new output directory
for PIPS_name, output_parsivel_filepath in zip(PIPS_names, output_parsivel_filepaths):
    print("Saving {}".format(output_parsivel_filepath))
    parsivel_ds_dict[PIPS_name].to_netcdf(output_parsivel_filepath)

In [ ]:
# Now do the same for the conventional dataset

conv_var_units_dict = {
    'pt': 'degrees Kelvin',
    'qv': 'kg per kg (unitless)',
    'rho': 'kg per cubic meter'
}

In [ ]:
# Now go through and update the units attribute for the needed variables using this dictionary

for PIPS_name in PIPS_names:
    for var_name in conv_ds_dict[PIPS_name].data_vars:
        if var_name in conv_var_units_dict:
            unit = conv_var_units_dict[var_name]
            conv_ds_dict[PIPS_name][var_name].attrs['units'] = unit
            

In [ ]:
# Now save to new output directory
for PIPS_name, output_conv_filepath in zip(PIPS_names, output_conv_filepaths):
    print("Saving {}".format(output_conv_filepath))
    conv_ds_dict[PIPS_name].to_netcdf(output_conv_filepath)